# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [18]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices

In [19]:
df = pd.read_csv('previsao_de_renda.csv')
df.drop(columns='Unnamed: 0', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   id_cliente             15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  object 
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          12427 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(6)
memory usa

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [23]:
#1
y, X = dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia', data=df, return_type='dataframe')
modelo = sm.OLS(y, X)
resultado = modelo.fit()
print(resultado.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Tue, 05 Dec 2023   Prob (F-statistic):               0.00
Time:                        23:43:05   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

- Os resultados indicam que algumas variáveis, como sexo, posse de veículo, posse de imóvel e tipo de renda para empresários e servidores públicos, têm uma associação significativa com o logaritmo da renda enquanto outras variáveis, como nível de educação e tipo de renda para bolsistas e pensionistas, não mostraram significância estatística. 

- Isso sugere que essas variáveis podem não ter um impacto forte ou consistente na renda dos indivíduos na amostra analisada.

In [24]:
#2
y, X = dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia', data=df, return_type='dataframe')
X = X.drop(columns='educacao[T.Pós graduação]')
modelo_atualizado = sm.OLS(y, X)
resultado_atualizado = modelo_atualizado.fit()
print(resultado_atualizado.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     300.0
Date:                Tue, 05 Dec 2023   Prob (F-statistic):               0.00
Time:                        23:43:10   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12403   BIC:                         2.736e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

- O modelo inicial era mais robusto e explicativo para a previsão do logaritmo da renda. 

- A remoção da variável 'educacao[T.Pós graduação]' não melhorou o modelo e na verdade resultou em uma perda significativa de poder explicativo.

In [26]:
#3
y, X = dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia', data=df, return_type='dataframe')
X = X.drop(columns=['educacao[T.Pós graduação]', 'educacao[T.Secundário]'])
modelo_atualizado = sm.OLS(y, X)
resultado_atualizado = modelo_atualizado.fit()
print(resultado_atualizado.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     313.6
Date:                Tue, 05 Dec 2023   Prob (F-statistic):               0.00
Time:                        23:52:57   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12404   BIC:                         2.735e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

- A remoção das variáveis menos significativas com base apenas no p-valor resultou em uma perda significativa de poder explicativo. 

- Isso sugere que, embora a simplificação do modelo seja uma prática comum para evitar o overfitting, é essencial considerar o equilíbrio entre simplicidade e capacidade explicativa do modelo.